In [2]:
!pip install datasets transformers huggingface_hub torchaudio librosa jiwer accelerate torch evaluate audiomentations

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
from datasets import load_dataset, DatasetDict

dataset = load_dataset("hr16/ViSpeech-Gender-Dialect-Classification", split='test', trust_remote_code=True)


Generating train split:   0%|          | 0/11752 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2982 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4978 [00:00<?, ? examples/s]

In [5]:
dataset

Dataset({
    features: ['audio', 'gender', 'dialect'],
    num_rows: 4978
})

In [6]:
dialect_mapping = {
    'northern dialect': 'Northern',
    'southern dialect': 'Southern',
    'central dialect': 'Central',
    'highland central dialect': 'Central',
    'minority ethnic group dialect': None  # Mark for removal
}

# Function to map dialects and filter out 'minority ethnic group dialect'
def map_and_filter_dialects(example):
    mapped_dialect = dialect_mapping.get(example['dialect'], None)
    if mapped_dialect:
        example['dialect'] = mapped_dialect
        return example
    return None

# Apply the mapping and filtering
dataset = dataset.filter(map_and_filter_dialects)
dataset = dataset.map(lambda x: {'dialect': dialect_mapping[x['dialect']]})

Filter:   0%|          | 0/4978 [00:00<?, ? examples/s]

Map:   0%|          | 0/4978 [00:00<?, ? examples/s]

In [7]:
from datasets import load_dataset, DatasetDict, Dataset

# Function to stratify split dataset by dialect
def stratified_split(dataset, train_size=0.8, val_size=0.1, test_size=0.1):
    assert train_size + val_size + test_size == 1.0, "The sum of train_size, val_size, and test_size should be 1.0"

    # Group by dialect
    dialects = dataset.unique('dialect')
    split_data = {'train': [], 'validation': [], 'test': []}

    for dialect in dialects:
        # Filter dataset by dialect
        dialect_dataset = dataset.filter(lambda x: x['dialect'] == dialect)
        
        # Split the dialect dataset into train and remaining (val+test)
        train_test_split_data = dialect_dataset.train_test_split(test_size=1 - train_size)
        train_dataset = train_test_split_data['train']
        remaining_dataset = train_test_split_data['test']

        # Split the remaining dataset into validation and test
        val_test_split_data = remaining_dataset.train_test_split(test_size=test_size / (val_size + test_size))
        val_dataset = val_test_split_data['train']
        test_dataset = val_test_split_data['test']

        # Add to the overall split_data
        split_data['train'].append(train_dataset)
        split_data['validation'].append(val_dataset)
        split_data['test'].append(test_dataset)

    # Concatenate the split data for each dialect
    split_data['train'] = Dataset.from_dict({k: sum([d[k] for d in split_data['train']], []) for k in split_data['train'][0].features})
    split_data['validation'] = Dataset.from_dict({k: sum([d[k] for d in split_data['validation']], []) for k in split_data['validation'][0].features})
    split_data['test'] = Dataset.from_dict({k: sum([d[k] for d in split_data['test']], []) for k in split_data['test'][0].features})

    return DatasetDict(split_data)

# Perform the stratified split
dataset = stratified_split(dataset)

# Display the new dataset splits
print(dataset)

Filter:   0%|          | 0/4978 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4978 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4978 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['audio', 'gender', 'dialect'],
        num_rows: 3982
    })
    validation: Dataset({
        features: ['audio', 'gender', 'dialect'],
        num_rows: 497
    })
    test: Dataset({
        features: ['audio', 'gender', 'dialect'],
        num_rows: 499
    })
})


In [8]:
dataset = dataset.remove_columns(['gender'])

In [10]:
from collections import Counter

# Function to count the number of samples in each split
def count_samples_in_splits(dataset):
    counts = {}
    for split_name, split_data in dataset.items():
        dialect_counts = Counter([sample['dialect'] for sample in split_data])
        counts[split_name] = dialect_counts
    return counts

# Count the number of samples in each split
counts = count_samples_in_splits(dataset)

# Print the distribution of dialects in each split
for split_name, dialect_counts in counts.items():
    print(f"{split_name.capitalize()} set distribution: {dict(dialect_counts)}")


Train set distribution: {'Northern': 2150, 'Southern': 1268, 'Central': 564}
Validation set distribution: {'Northern': 269, 'Southern': 158, 'Central': 70}
Test set distribution: {'Northern': 269, 'Southern': 159, 'Central': 71}


In [11]:
# Định nghĩa danh sách các nhãn phân loại
labels_list = sorted(dataset["train"].unique("dialect"))

def id2label(label_id):
    return labels_list[label_id]

# Tạo hàm label2id để chuyển đổi từ nhãn sang id
label2id = {label: i for i, label in enumerate(labels_list)}
id2label = {v: k for k, v in label2id.items()}

print("Id corresponding to label:", label2id)
print("Label corresponding to id:", id2label)

Id corresponding to label: {'Central': 0, 'Northern': 1, 'Southern': 2}
Label corresponding to id: {0: 'Central', 1: 'Northern', 2: 'Southern'}


In [12]:
len(label2id)

3

In [13]:
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base")
feature_extractor

preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": false,
  "sampling_rate": 16000
}

In [14]:
import random
import numpy as np
from IPython.display import Audio, display

for _ in range(3):
    rand_idx = random.randint(0, len(dataset["validation"])-1)
    example = dataset["validation"][rand_idx]
    audio = example["audio"]

    audio_array = np.array(audio["array"])  # Convert to ndarray

    print(f'Label: {example["dialect"]}')
    print(f'Shape: {audio_array.shape}, sampling rate: {audio["sampling_rate"]}')
    display(Audio(audio_array, rate=audio["sampling_rate"]))
    print()


Label: Northern
Shape: (151460,), sampling rate: 16000



Label: Southern
Shape: (80000,), sampling rate: 16000



Label: Northern
Shape: (83807,), sampling rate: 16000


In [15]:
import random
from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift, Shift
import numpy as np

# Define audio augmentation pipeline
augment = Compose([
    AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.01, p=0.5),
    TimeStretch(min_rate=0.8, max_rate=1.1, p=0.5),
    PitchShift(min_semitones=-3, max_semitones=3, p=0.5),
    Shift(min_shift=-0.3, max_shift=0.3, p=0.5),
])

# Function to augment audio
def augment_audio(batch):
    audio = batch["audio"]
    samples = np.array(audio["array"])
    sample_rate = audio["sampling_rate"]
    
    augmented_samples = augment(samples=samples, sample_rate=sample_rate)
    
    batch["audio"] = {
        "array": augmented_samples,
        "sampling_rate": sample_rate
    }
    return batch

In [17]:
dataset = dataset.map(augment_audio)

Map:   0%|          | 0/3982 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/audiomentations/core/transforms_interface.py:62: UserWarning: Warning: input samples dtype is np.float64. Converting to np.float32
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/audiomentations/core/transforms_interface.py:62: UserWarning: Warning: input samples dtype is np.float64. Converting to np.float32
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/audiomentations/core/transforms_interface.py:62: UserWarning: Warning: input samples dtype is np.float64. Converting to np.float32
  warnings.warn(


Map:   0%|          | 0/497 [00:00<?, ? examples/s]

Map:   0%|          | 0/499 [00:00<?, ? examples/s]

In [18]:
for _ in range(3):
    rand_idx = random.randint(0, len(dataset["validation"])-1)
    example = dataset["validation"][rand_idx]
    audio = example["audio"]
    
    audio_array = np.array(audio["array"])  # Convert to ndarray

    print(f'Label: {example["dialect"]}')
    print(f'Shape: {audio_array.shape}, sampling rate: {audio["sampling_rate"]}')
    display(Audio(audio_array, rate=audio["sampling_rate"]))
    print()

Label: Northern
Shape: (94161,), sampling rate: 16000



Label: Northern
Shape: (18409,), sampling rate: 16000



Label: Northern
Shape: (66166,), sampling rate: 16000


In [19]:
max_duration = 20.0  # seconds

def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays,
        sampling_rate=feature_extractor.sampling_rate,
        max_length=int(feature_extractor.sampling_rate * max_duration),
        truncation=True,
    )
    inputs["labels"] = [label2id[label] for label in examples["dialect"]]
    return inputs

In [20]:
for split in ['train', 'validation', 'test']:
    dataset[split] = dataset[split].map(preprocess_function, remove_columns=dataset[split].column_names, batched=True)

Map:   0%|          | 0/3982 [00:00<?, ? examples/s]

2024-08-12 09:41:58.786731: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-12 09:41:58.786855: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-12 09:41:58.916077: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Map:   0%|          | 0/497 [00:00<?, ? examples/s]

Map:   0%|          | 0/499 [00:00<?, ? examples/s]

In [21]:
dataset['train']

Dataset({
    features: ['input_values', 'labels'],
    num_rows: 3982
})

In [22]:
import evaluate

accuracy = evaluate.load("accuracy")

In [23]:
import numpy as np

def compute_metrics(eval_pred):
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=eval_pred.label_ids)

In [24]:
from transformers import AutoModelForAudioClassification, TrainingArguments, Trainer

num_labels = len(id2label)
model = AutoModelForAudioClassification.from_pretrained(
    "facebook/wav2vec2-base",
    attention_dropout=0.05,
    hidden_dropout=0.0,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.05,
    num_labels=num_labels,
    label2id=label2id,
    id2label=id2label
)

/opt/conda/lib/python3.10/site-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/380M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [25]:
!wandb login edfa6bb2b01b5b2e74dd9f112d7ba0da853ebd0e

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [30]:
training_args = TrainingArguments(
    output_dir="test-model",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=8,
    gradient_checkpointing=True,
    evaluation_strategy="epoch",
    num_train_epochs=10,
    weight_decay=0.01,
    fp16=True,
    warmup_ratio=0.3,
    logging_steps=62,
    dataloader_num_workers=4,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=True,
    report_to=['wandb']
)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [31]:
def create_dataloader(dataset, batch_size, shuffle=True):
    return DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        num_workers=4,
        pin_memory=True,
        prefetch_factor=2  # Prefetch 2 batches
    )

In [32]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
)


In [33]:
trainer.train()

/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
0,1.055500,0.960131,0.579477
1,0.886200,0.806399,0.688129
2,0.733100,0.638653,0.770624
4,0.540300,0.557736,0.792757
5,0.463000,0.416222,0.875252
6,0.408100,0.525530,0.839034
8,0.328700,0.340332,0.909457
9,0.311600,0.338903,0.903421


/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWa

TrainOutput(global_step=620, training_loss=0.5745973925436697, metrics={'train_runtime': 4364.8068, 'train_samples_per_second': 9.123, 'train_steps_per_second': 0.142, 'total_flos': 3.833227239812147e+18, 'train_loss': 0.5745973925436697, 'epoch': 9.959839357429718})

In [34]:
trainer.push_to_hub()


CommitInfo(commit_url='https://huggingface.co/Thienpkae/test-model/commit/3fa7007befc9bf0d7c009394e9e3ae6609193c9f', commit_message='End of training', commit_description='', oid='3fa7007befc9bf0d7c009394e9e3ae6609193c9f', pr_url=None, pr_revision=None, pr_num=None)

In [35]:
trainer.save_model("./saved_model")

In [38]:
trainer.evaluate(dataset['test'])

/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


{'eval_loss': 0.3656424283981323,
 'eval_accuracy': 0.9038076152304609,
 'eval_runtime': 20.6871,
 'eval_samples_per_second': 24.121,
 'eval_steps_per_second': 3.045,
 'epoch': 9.959839357429718}

In [49]:
# Save the model and tokenizer/feature extractor
model.save_pretrained("test-model")
feature_extractor.save_pretrained("test-model")

['test-model/preprocessor_config.json']

In [51]:
from transformers import AutoModelForAudioClassification, AutoFeatureExtractor

# Load the model
model = AutoModelForAudioClassification.from_pretrained("Thienpkae/test-model")

# Load the feature extractor
feature_extractor = AutoFeatureExtractor.from_pretrained("Thienpkae/test-model")


In [ ]:
from sklearn.metrics import classification_report

# Make predictions on the test dataset
test_results = trainer.predict(dataset["test"])

# Extract predictions and labels
pred_labels = np.argmax(test_results.predictions, axis=1)
true_labels = dataset["test"]["labels"]

# Compute and print classification report
report = classification_report(true_labels, pred_labels, target_names=['Central', 'Northern', 'Southern'])
print(report)


In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Compute confusion matrix
cm = confusion_matrix(true_labels, pred_labels, labels=[0, 1, 2]) 

# Display confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Central', 'Northern', 'Southern'])
disp.plot(cmap=plt.cm.Blues)
plt.show()
